In [1]:
import sys
import json
from os import getenv
from pathlib import Path
from datetime import datetime
from openai import OpenAI

# Set the base path
base_path = Path("../../")  # One level up from the current working directory

# Add the src/ directory to sys.path using base_path
sys.path.append(str((base_path / "src").resolve()))

from tools.python_math_executor import (
    PYTHON_MATH_EXECUTION_TOOL,
    execute_python
)
from tool_calling import run_tool_call_loop
from save_outputs import save_output

In [4]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=getenv("OPENROUTER_API_KEY")
)

# Messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Calculate the probability using Bayes' theorem:\n"
                                "P(A) = 0.01\n"
                                "P(B|A) = 0.9\n"
                                "P(B) = 0.05"}
]

# Tool registration
tools = [
    PYTHON_MATH_EXECUTION_TOOL
]

TOOL_MAPPING = {
    "execute_python": execute_python
}

MODEL = "google/gemini-2.5-pro-preview"

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # To allow for multiple runs

In [5]:
response, updated_messages = run_tool_call_loop(
    openai_client=client,
    model=MODEL,
    tools=tools,
    messages=messages,
    tool_mapping=TOOL_MAPPING,
    verbose=True
)


--- Iteration 1 ---
🛠️  Model requested tool: execute_python with args: {'code': 'P_A = 0.01\nP_B_given_A = 0.9\nP_B = 0.05\nP_A_given_B = (P_B_given_A * P_A) / P_B'}

--- Iteration 2 ---
🔚 Final model output:
 The probability of A given B, P(A|B), is 0.18.


In [6]:
save_output(
    response=response,
    messages=messages,
    base_path=base_path / "results" / "tool_calling_python_math",
    timestamp=timestamp,
    dir_name=MODEL
)

✅ Final response saved to: ../../results/tool_calling_python_math/2025-05-18_13-15-25/gpt-4o-mini/final_response.txt
✅ Message history saved to: ../../results/tool_calling_python_math/2025-05-18_13-15-25/gpt-4o-mini/message_history.json
